### Web Pricing

In [25]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import win32com.client as win32

products = pd.read_excel(r'Products.xlsx')
display(products)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,Iphone 11,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,NaN,NaN
1,Galaxy S21,https://www.amazon.com.br/SM-G996B-Standard-Du...,https://www.americanas.com.br/produto/28752403...,https://www.magazineluiza.com.br/smartphone-sa...,7000,NaN,NaN


In [26]:
def text_transform(text):
    return float(text.replace('R$', '').replace('.', '').replace(',', '.'))

In [29]:
driver = webdriver.Chrome()

sendemail = False

discount_min = 0.2

for i, rows in products.iterrows():
    # Get Amazon Price
    driver.get(rows['Amazon'])
    try:
        price_amz = driver.find_element(By.XPATH, '//*[@id="corePrice_desktop"]/div/table/tbody/tr[1]/td[2]/span[1]/span[2]').text
        price_amz = text_transform(price_amz)
    except:
        try:
            price_amz = driver.find_element(By.XPATH, '//*[@id="olp_feature_div"]/div[2]/span[1]/a/span[2]').text
            price_amz = text_transform(price_amz)
        except:
            print('Product {} not avilabe on Amazon'.format(rows['Link Produto']))
            price_amz = rows['Preço Original'] * 3
    

    # Get Lojas Americanas Price
    driver.get(rows['Lojas Americanas'])
    try:
        price_lam = driver.find_element(By.CLASS_NAME, 'src__BestPrice-sc-1jvw02c-5').text
        price_lam = text_transform(price_lam)
    except:
        print('Product {} not avilabe on B2W'.format(rows['Link Produto']))
        price_lam = rows['Preço Original'] * 3        

    # Get Magalu Price
    driver.get(rows['Magazine Luiza'])
    try:
        price_mgl = driver.find_element(By.CLASS_NAME, 'price-template__text').text
        price_mgl = text_transform(price_mgl)
    except:
        print('Product {} not avilabe on MAGALU'.format(rows['Link Produto']))
        price_mgl = rows['Preço Original'] * 3 
        

    print(price_amz, price_lam, price_mgl)
    
    original_price = rows['Preço Original']
    
    lst_price = [(price_amz, 'Amazon'), (price_lam, 'Lojas Americanas'), (price_mgl, 'Magazine Luiza'), (original_price, 'Original')]
    lst_price.sort()
    
    products.loc[i, 'Preço Atual'] = lst_price[0][0]
    products.loc[i, 'Local'] = lst_price[0][1]
    
    if lst_price[0][0] <= original_price * (1-discount_min):
        sendemail = True
        

4566.0 3905.07 3905.07
6199.0 4048.7 7199.1


In [30]:
display(products)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,Iphone 11,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,3905.07,Lojas Americanas
1,Galaxy S21,https://www.amazon.com.br/SM-G996B-Standard-Du...,https://www.americanas.com.br/produto/28752403...,https://www.magazineluiza.com.br/smartphone-sa...,7000,4048.70,Lojas Americanas


In [34]:
# save file
products.to_excel('Products.xlsx')

In [ ]:
# send e-Mail
outlook = win32.Dispatch('outlook.application')

if sendemail:
    mail = outlook.CreateItem(0)
    mail.To = 'destiny@mail.com'
    mail.Subject = f'Found product with {discount_min:.0%} or more Discount'
    
    # Filtering product table
    table_filtered = products.loc[products['Preço Atual'] <= products['Preço Original']*(1-discount_min), :]
    
    
    mail.HTMLBody = f'<p>Products with {discount_min:.0%} or more</p>{table_filtered.to_html()}'
    
    
    
    mail.Send()
    
print ('End of Analisys')
driver.quit()